In [1]:
import findspark
findspark.init('/home/nowshad/spark-3.3.2-bin-hadoop3/')
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('titatic').getOrCreate()

23/03/19 21:51:34 WARN Utils: Your hostname, nowshad resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/19 21:51:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 21:51:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/19 21:51:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df=spark.read.csv('/home/nowshad/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv',inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols=df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [8]:
my_final_data=my_cols.na.drop()

In [10]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder,StringIndexer)

In [11]:
gender_indexer=StringIndexer(inputCol='Sex',outputCol='SexIndex')

In [12]:
# A B C
# 0 1 2
#one Hot Encode
#key A B C
#Example A
[1, 0, 0]
gender_encoder=OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [13]:
embark_indexer=StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder=OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [14]:
assembler=VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                         outputCol='features')

In [16]:
from pyspark.ml.classification import LogisticRegression

In [19]:
from pyspark.ml import Pipeline

In [20]:
log_reg_titanic=LogisticRegression(featuresCol='features',labelCol='Survived')

In [22]:
pipeline=Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [23]:
train_data,test_data=my_final_data.randomSplit([0.7,0.3])

In [26]:
fit_model=pipeline.fit(train_data)

In [27]:
results=fit_model.transform(test_data)

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
my_evel=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [31]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [33]:
AUC=my_evel.evaluate(results)

In [34]:
AUC

0.7920965364775238